In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [16]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
# Preprocess the data

data=data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [18]:
# Encode categorical variables

label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [19]:
# One-hot encode the 'Geography' column

onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [20]:
# Combine one-hot encoded columns with the original data
data=pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [21]:
# Split the data into features and target variable
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Scale the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [22]:
# Save the encoders and scalers for later use

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

## ANN regression Problem Statement

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
# Build the neural network model
model=Sequential(
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1) ## Output layer for regression
    ]
)

/Users/sushrinadhakal/.pyenv/versions/3.10.12/envs/tf-env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
## Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up tensorboard logging
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [28]:
# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [29]:
# Train the model

history= model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 101119.6016 - mae: 101119.6016 - val_loss: 98542.2969 - val_mae: 98542.2969
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - loss: 99798.4219 - mae: 99798.4219 - val_loss: 97152.9844 - val_mae: 97152.9844
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - loss: 99297.0859 - mae: 99297.0859 - val_loss: 93563.2734 - val_mae: 93563.2734
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step - loss: 93677.1016 - mae: 93677.1016 - val_loss: 87529.1172 - val_mae: 87529.1172
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - loss: 88753.1172 - mae: 88753.1172 - val_loss: 79624.9844 - val_mae: 79624.9844
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - loss: 79260.6250 - mae: 79260.6250 - val_loss: 71120.0938 - val_mae: 71120.0938
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - loss: 70209.1016 - mae: 70209.1016 - val_loss: 63326.4961 - val_mae: 63326.4961
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [31]:
%load_ext tensorboard

In [34]:
%tensorboard --logdir regressionlogs/fit/20250530-221919

Reusing TensorBoard on port 6007 (pid 90510), started 0:00:13 ago. (Use '!kill 90510' to kill it.)

In [35]:
# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step - loss: 51079.4688 - mae: 51079.4688
Test Loss: 50223.16796875, Test MAE: 50223.16796875


In [36]:
model.save('regression_model.h5')